In [3]:
import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import folium
from collections import Counter
from ipynb.fs.defs.Local_search import get_row
from ipynb.fs.defs.Local_search import getName



In [ ]:
df = pd.read_csv("../Dati/request_cr.csv", delimiter=",", encoding = "ISO-8859-1")
to_ = list(df['biblioteca_destinazione'])
from_ = list(df['biblioteca_partenza'])
data = np.column_stack((from_, to_)).tolist()
occ = [[x, data.count(x)] for x in data]

In [36]:
def unique(list_): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list_: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list
unique_occ = unique(occ)

In [44]:
origins = []
destinations = []
values = []
for i in unique_occ:
    origins.append(i[0][0])
    destinations.append(i[0][1])
    values.append(i[1])
d = {'origin' : origins, 'destination': destinations, 'num_orders': values}
dataframe = pd.DataFrame(d)
dataframe = dataframe.sort_values('num_orders', ascending = False)


,origin,destination,num_orders
135,BRESCIA,CREMA,20176
75,BRESCIA,OFFANENGO,3958
152,CREMA,BRESCIA,2540
67,BRESCIA,SONCINO,2533
226,RIVOLTA D'ADDA,CREMA,1965
66,BRESCIA,SORESINA,1936
38,BRESCIA,RIVOLTA D'ADDA,1723
207,RIVOLTA D'ADDA,BRESCIA,1641
137,CASTELLEONE,CREMA,1582
93,BRESCIA,MADIGNANO,1566


In [46]:
dataframe.to_csv("../Dati/num_orders.csv", index = False, sep = ';')

In [41]:
dataframe = pd.read_csv("../Dati/num_orders.csv", delimiter=";", encoding = "ISO-8859-1")
coor = pd.read_csv('../Dati/altro/CitiesLocation.csv', delimiter=";", encoding = "ISO-8859-1", index_col = 0)
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
orig = []
dest = []
exceptions_orig = []
exceptions_dest = []
for i in range(len(dataframe)):
    try:
        orig.append(getName(dft, dataframe.iloc[i]['origin']))
    except:
        exceptions_orig.append(dataframe.iloc[i]['origin'])
    try:
        dest.append(getName(dft, dataframe.iloc[i]['destination']))
    except:
        exceptions_dest.append(dataframe.iloc[i]['destination'])
    


In [43]:
print(set(exceptions_orig))
print(set(exceptions_dest))
print(len(set(exceptions_orig)))
print(len(set(exceptions_dest)))

{'SAN DANIELE PO', 'UNIONE MUNICIPIA', 'PIADENA DRIZZONA', 'Cremona-A.R.T.E.', 'TORRICELLA DEL PIZZO', 'BRESCIA', 'CENTRO FUMETTO A. PAZIENZA', 'CASALE CREMASCO VIDOLASCO', 'CASALETTO CEREDANO', 'CREMONA. LA PICCOLA BIBLIOTECA', 'Cremona-M.I.T.O.', 'UFF. BIB. CREMONA', "CORTE DE' FRATI", 'Cremona - Manin (scolastica)', 'GADESCO PIEVE DELMONA', 'CREMOSANO', 'Cremona - Anguissola (scolastica)', 'CREMONA - CDA', 'GENIVOLTA'}
{'SAN DANIELE PO', 'UNIONE MUNICIPIA', 'PIADENA DRIZZONA', 'Cremona-A.R.T.E.', 'TORRICELLA DEL PIZZO', 'BRESCIA', 'CENTRO FUMETTO A. PAZIENZA', 'CASALE CREMASCO VIDOLASCO', 'CASALETTO CEREDANO', 'CREMONA. LA PICCOLA BIBLIOTECA', 'Cremona-M.I.T.O.', 'UFF. BIB. CREMONA', "CORTE DE' FRATI", 'Cremona - Manin (scolastica)', 'GADESCO PIEVE DELMONA', 'CREMOSANO', 'Cremona - Anguissola (scolastica)', 'CREMONA - CDA', 'GENIVOLTA', 'CALVATONE'}
19
20
